In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
# from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
from datetime import datetime, timedelta
import requests
import json

In [4]:
ip='220.78.231.223'

In [5]:
cluster = '118.34.234.79:7077'
spark = SparkSession \
        .builder \
        .master('local[2]') \
        .config("spark.driver.extraClassPath", "/Users/mdgome/Downloads/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar") \
        .appName('tempera_obs') \
        .getOrCreate()

22/02/27 01:57:49 WARN Utils: Your hostname, Jungminui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.10 instead (on interface en0)
22/02/27 01:57:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/27 01:57:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# obs_post_data = pd.read_csv("./관측소 정보.csv")
obs_post_data= spark.read.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='obs_info',
    user='mdgome',
    password='Rlawjdals1!').load()
obs_post_data.show(5)

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


+----------+---------+----------+----------------------------------+-----------+-------------+
| data_type|  obs_lat|   obs_lon|                        obs_object|obs_post_id|obs_post_name|
+----------+---------+----------+----------------------------------+-----------+-------------+
|조위관측소|  36.6775|129.453056|조위,수온,염분,기온,기압,풍속,풍향|    DT_0011|         후포|
|조위관측소|35.024178|128.810933|조위,수온,염분,기온,기압,풍속,풍향|    DT_0063|       가덕도|
|조위관측소|34.684167|125.435556|조위,수온,염분,기온,기압,풍속,풍향|    DT_0035|       흑산도|
|조위관측소|37.731944|126.522222|                         조위,기압|    DT_0032|     강화대교|
|조위관측소|34.028333|127.308889|조위,수온,염분,기온,기압,풍속,풍향|    DT_0031|       거문도|
+----------+---------+----------+----------------------------------+-----------+-------------+
only showing top 5 rows



In [9]:
import pprint
pprint.pprint(spark.sparkContext.getConf().getAll())
# spark.stop()

[('spark.driver.host', '172.30.1.10'),
 ('spark.app.id', 'local-1645894671269'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '50656'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.extraClassPath',
  '/Users/mdgome/Downloads/mysql-connector-java-8.0.28/mysql-connector-java-8.0.28.jar'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.startTime', '1645894669946'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.name', 'tempera_obs'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.master', 'local[2]'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mdgome/workspace/study/StudyDataEngineering/playadata/middleproject/spark-warehouse')]


In [8]:
obs_post_list = obs_post_data.select('obs_post_id').rdd.flatMap(lambda x: x).collect()
print(list(obs_post_list)) # 디버깅 용

['DT_0011', 'DT_0063', 'DT_0035', 'DT_0032', 'DT_0031', 'DT_0042', 'DT_0029', 'DT_0041', 'DT_0039', 'DT_0058', 'IE_0060', 'DT_0026', 'DT_0049', 'IE_0062', 'DT_0018', 'IE_0061', 'DT_0017', 'TW_0089', 'HB_0007', 'DT_0065', 'HB_0008', 'DT_0057', 'HB_0009', 'DT_0062', 'TW_0079', 'DT_0023', 'DT_0007', 'TW_0080', 'TW_0081', 'DT_0006', 'TW_0082', 'DT_0025', 'TW_0083', 'DT_0005', 'TW_0084', 'DT_0056', 'TW_0092', 'DT_0061', 'SF_0001', 'DT_0010', 'TW_0078', 'DT_0051', 'TW_0087', 'DT_0022', 'TW_0090', 'DT_0012', 'TW_0074', 'DT_0008', 'HB_0001', 'DT_0067', 'HB_0002', 'DT_0037', 'HB_0003', 'DT_0016', 'TW_0062', 'DT_0092', 'TW_0069', 'DT_0003', 'TW_0070', 'DT_0044', 'TW_0072', 'DT_0043', 'TW_0077', 'DT_0027', 'TW_0075', 'DT_0013', 'TW_0076', 'DT_0020', 'DT_0068', 'TW_0088', 'TW_0085', 'DT_0001', 'TW_0091', 'DT_0052', 'TW_0086', 'DT_0024', 'SF_0005', 'DT_0004', 'TW_0093', 'DT_0028', 'TW_0094', 'DT_0021', 'DT_0050', 'KG_0021', 'DT_0014', 'KG_0024', 'KG_0025', 'DT_0002', 'KG_0028', 'DT_0091', 'KG_0101'

In [14]:
service_key="1LJG14JWnAylzKi5GvmuGg=="
now_datetime=datetime.now().strftime('%Y%m%d')
now_datetime # 디버깅 용

'20220225'

In [518]:
yesterday = datetime.today() - timedelta(days = 1 )
yesterday = yesterday.strftime('%Y%m%d')
yesterday

'20220224'

In [519]:
# for obs in obs_post_list:
#     print(obs)
#     url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
#     url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
#     url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
#     url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey="+service_key+"&ObsCode="+obs+"&Date="+now_datetime+"&ResultType=json"
#     response = requests.get(url).json()
#     response1 = requests.get(url1).json()
#     response2 = requests.get(url2).json()
#     response3 = requests.get(url3).json()
#     df = spark.createDataFrame(response["result"]["data"])
#     df1 = spark.createDataFrame(response1["result"]["data"])
#     df2 = spark.createDataFrame(response2["result"]["data"])
#     df3 = spark.createDataFrame(response3["result"]["data"])
    
#     output_df = df.join(df1,'record_time')#.select(df1.record_time, df1.air_pres, df.air_temp)
#     output_df = output_df.join(df2, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,df2.wind_dir,df2.wind_speed)
#     output_df = output_df.join(df3, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
#     output_df = output_df.withColumn('id',lit(obs))
    
#     output_df.write.csv('./'+now_datetime+"/"+obs+".csv")
#     # MySQL Data 적재 하는 부분 생성
#     # output_df.write.format('jdbc').options(
#     #     url='jdbc:mysql://'+ip+'/weather',
#     #     driver='com.mysql.jdbc.Driver',
#     #     dbtable=response["result"]["meta"]["obs_post_id"],
#     #     user='mdgome',
#     #     password='Rlawjdals1!').mode('append').save()
    

In [520]:
# 밑에 부분은 테스트 코드에요

In [521]:
# url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"
# response1=requests.get(url1).json()
# response1["result"]["data"]

In [522]:
# # 기온 api
# url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"
# # 기압 api
# url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"
# # 풍속 api
# url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"
# # 염분 api
# url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"
# # 조위 실측/예측 api
# url4="http://www.khoa.go.kr/api/oceangrid/tideCurPre/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+now_datetime+"&ResultType=json"

In [523]:
# 기온 api
url="http://www.khoa.go.kr/api/oceangrid/tideObsAirTemp/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 기압 api
url1="http://www.khoa.go.kr/api/oceangrid/tideObsAirPres/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 풍속 api
url2="http://www.khoa.go.kr/api/oceangrid/tideObsWind/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 염분 api
url3="http://www.khoa.go.kr/api/oceangrid/tideObsSalt/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 조위 실측/예측 api
url4="http://www.khoa.go.kr/api/oceangrid/tideCurPre/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"
# 수온 api
url5="http://www.khoa.go.kr/api/oceangrid/tideObsTemp/search.do?ServiceKey="+service_key+"&ObsCode=DT_0004&Date="+yesterday+"&ResultType=json"

In [524]:
response = requests.get(url).json()
response1 = requests.get(url1).json()
response2 = requests.get(url2).json()
response3 = requests.get(url3).json()
response4 = requests.get(url4).json()
response5 = requests.get(url5).json()

In [525]:
# response["result"]["data"]
# # response["result"]["meta"]

In [526]:
# response1["result"]["data"]

In [527]:
# response2["result"]["data"]

In [528]:
# response3["result"]["data"]

In [529]:
# response4["result"]["data"]

In [530]:
# response5["result"]["data"]

In [531]:
df = spark.createDataFrame(response["result"]["data"])
df1 = spark.createDataFrame(response1["result"]["data"])
df2 = spark.createDataFrame(response2["result"]["data"])
df3 = spark.createDataFrame(response3["result"]["data"])
df4 = spark.createDataFrame(response4["result"]["data"])
df5 = spark.createDataFrame(response5["result"]["data"])

In [532]:
# df4.show(100)

In [533]:
output_df = df.join(df1,'record_time')#.select(df1.record_time, df1.air_pres, df.air_temp)
output_df = output_df.join(df2, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,df2.wind_dir,df2.wind_speed)
output_df = output_df.join(df3, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
output_df = output_df.join(df5, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,df3.salinity)
# output_df = output_df.join(df4, 'record_time')#.select(output_df.record_time,output_df.air_pres,output_df.air_temp,output_df.wind_dir,output_df.wind_speed,output_df.salinity,df4.pre_value,df4.real_value)

In [534]:
output_df = output_df.sort(asc("record_time"))
output_df.show(10)

+-------------------+--------+--------+--------+----------+--------+----------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|
+-------------------+--------+--------+--------+----------+--------+----------+
|2022-02-24 00:00:00|     1.3|  1032.9|   351.0|       3.6|    32.3|        13|
|2022-02-24 00:01:00|     1.3|  1033.1|   351.0|       2.7|    32.3|        13|
|2022-02-24 00:02:00|     1.4|  1032.9|   344.0|       5.0|    32.3|        13|
|2022-02-24 00:03:00|     1.3|  1033.1|     0.0|       5.0|    32.3|        13|
|2022-02-24 00:04:00|     1.3|  1032.9|   336.0|       3.8|    32.3|        13|
|2022-02-24 00:05:00|     1.4|  1033.1|   353.0|       3.1|    32.3|        13|
|2022-02-24 00:06:00|     1.2|  1033.1|   351.0|       4.4|    32.3|        13|
|2022-02-24 00:07:00|     1.2|  1033.1|   350.0|       3.4|    32.3|        13|
|2022-02-24 00:08:00|     1.1|  1033.1|   348.0|       4.7|    32.3|        13|
|2022-02-24 00:09:00|     1.1|  1032.9| 

In [535]:
df4 = df4.sort(asc("record_time"))
df4.show(10)

+---------+----------+----------------+
|pre_value|real_value|     record_time|
+---------+----------+----------------+
|       97|        97|2022-02-24 00:00|
|      132|       130|2022-02-24 01:00|
|      166|       163|2022-02-24 02:00|
|      189|       187|2022-02-24 03:00|
|      199|       202|2022-02-24 04:00|
|      199|       203|2022-02-24 05:00|
|      189|       192|2022-02-24 06:00|
|      173|       178|2022-02-24 07:00|
|      152|       159|2022-02-24 08:00|
|      130|       135|2022-02-24 09:00|
+---------+----------+----------------+
only showing top 10 rows



In [536]:
output_df = output_df.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))
df4 = df4.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [537]:
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+----------+-------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|     id|
+-------------------+--------+--------+--------+----------+--------+----------+-------+
|2022-02-24 00:00:00|     1.3|  1032.9|   351.0|       3.6|    32.3|        13|DT_0004|
|2022-02-24 00:01:00|     1.3|  1033.1|   351.0|       2.7|    32.3|        13|DT_0004|
|2022-02-24 00:02:00|     1.4|  1032.9|   344.0|       5.0|    32.3|        13|DT_0004|
|2022-02-24 00:03:00|     1.3|  1033.1|     0.0|       5.0|    32.3|        13|DT_0004|
|2022-02-24 00:04:00|     1.3|  1032.9|   336.0|       3.8|    32.3|        13|DT_0004|
+-------------------+--------+--------+--------+----------+--------+----------+-------+
only showing top 5 rows



In [538]:
df4.show(5)

+---------+----------+----------------+-------+
|pre_value|real_value|     record_time|     id|
+---------+----------+----------------+-------+
|       97|        97|2022-02-24 00:00|DT_0004|
|      132|       130|2022-02-24 01:00|DT_0004|
|      166|       163|2022-02-24 02:00|DT_0004|
|      189|       187|2022-02-24 03:00|DT_0004|
|      199|       202|2022-02-24 04:00|DT_0004|
+---------+----------+----------------+-------+
only showing top 5 rows



In [539]:
output_df = output_df.withColumn('air_temp', output_df['air_temp'].cast(DoubleType()))\
    .withColumn('air_pres', output_df['air_pres'].cast(DoubleType()))\
    .withColumn('wind_dir', output_df['wind_dir'].cast(DoubleType()))\
    .withColumn('record_time', output_df['record_time'].cast("timestamp"))\
    .withColumn('salinity', output_df['salinity'].cast(DoubleType()))
    # .withColumn('pre_value', df4['pre_value'].cast(IntegerType())) \
    # .withColumn('real_value', df4['real_value'].cast(IntegerType()))

In [540]:
output_df.printSchema()

root
 |-- record_time: timestamp (nullable = true)
 |-- air_temp: double (nullable = true)
 |-- air_pres: double (nullable = true)
 |-- wind_dir: double (nullable = true)
 |-- wind_speed: string (nullable = true)
 |-- salinity: double (nullable = true)
 |-- water_temp: string (nullable = true)
 |-- id: string (nullable = false)



In [541]:
output_df.show(5)

+-------------------+--------+--------+--------+----------+--------+----------+-------+
|        record_time|air_temp|air_pres|wind_dir|wind_speed|salinity|water_temp|     id|
+-------------------+--------+--------+--------+----------+--------+----------+-------+
|2022-02-24 00:00:00|     1.3|  1032.9|   351.0|       3.6|    32.3|        13|DT_0004|
|2022-02-24 00:01:00|     1.3|  1033.1|   351.0|       2.7|    32.3|        13|DT_0004|
|2022-02-24 00:02:00|     1.4|  1032.9|   344.0|       5.0|    32.3|        13|DT_0004|
|2022-02-24 00:03:00|     1.3|  1033.1|     0.0|       5.0|    32.3|        13|DT_0004|
|2022-02-24 00:04:00|     1.3|  1032.9|   336.0|       3.8|    32.3|        13|DT_0004|
+-------------------+--------+--------+--------+----------+--------+----------+-------+
only showing top 5 rows



In [542]:
df4 = df4.withColumn('pre_value', df4['pre_value'].cast(IntegerType())) \
    .withColumn('record_time', df4['record_time'].cast("timestamp"))\
    .withColumn('real_value', df4['real_value'].cast(IntegerType()))

In [543]:
df4.printSchema()

root
 |-- pre_value: integer (nullable = true)
 |-- real_value: integer (nullable = true)
 |-- record_time: timestamp (nullable = true)
 |-- id: string (nullable = false)



In [544]:
df4.show(5)

+---------+----------+-------------------+-------+
|pre_value|real_value|        record_time|     id|
+---------+----------+-------------------+-------+
|       97|        97|2022-02-24 00:00:00|DT_0004|
|      132|       130|2022-02-24 01:00:00|DT_0004|
|      166|       163|2022-02-24 02:00:00|DT_0004|
|      189|       187|2022-02-24 03:00:00|DT_0004|
|      199|       202|2022-02-24 04:00:00|DT_0004|
+---------+----------+-------------------+-------+
only showing top 5 rows



In [545]:
output_df.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='DT_0004',
    user='mdgome',
    password='Rlawjdals1!').mode('append').save()

In [546]:
df4.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='DT_0004_tideCurPre',
    user='mdgome',
    password='Rlawjdals1!').mode('append').save()

In [114]:
# 수온 api
url = "http://www.khoa.go.kr/api/oceangrid/tidalBuTemp/search.do?ServiceKey="+service_key+"&ObsCode=KG_0021&Date="+now_datetime+"&ResultType=json"
# 유속/유향 api 
url1= "http://www.khoa.go.kr/api/oceangrid/tidalBu/search.do?ServiceKey="+service_key+"&ObsCode=KG_0021&Date="+now_datetime+"&ResultType=json"
url1

'http://www.khoa.go.kr/api/oceangrid/tidalBu/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ObsCode=KG_0021&Date=20220225&ResultType=json'

In [83]:
response = requests.get(url).json()
response1 = requests.get(url1).json()

In [84]:
response["result"]["data"]

[{'record_time': '2022-02-25 00:00:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 00:30:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 01:00:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 01:30:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 02:00:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 02:30:00', 'water_temp': '17.21'},
 {'record_time': '2022-02-25 03:00:00', 'water_temp': '17.16'},
 {'record_time': '2022-02-25 03:30:00', 'water_temp': '17.14'},
 {'record_time': '2022-02-25 04:00:00', 'water_temp': '17.11'},
 {'record_time': '2022-02-25 04:30:00', 'water_temp': '17.05'},
 {'record_time': '2022-02-25 05:00:00', 'water_temp': '17.04'},
 {'record_time': '2022-02-25 05:30:00', 'water_temp': '17.11'},
 {'record_time': '2022-02-25 06:00:00', 'water_temp': '17.12'},
 {'record_time': '2022-02-25 06:30:00', 'water_temp': '17.14'},
 {'record_time': '2022-02-25 07:00:00', 'water_temp': '17.1'},
 {'record_time': '2022-02-25 07:30:00', '

In [85]:
response1

{'result': {'data': [{'obs_time': '2022-02-25 00:00:00',
    'current_direct': '224.0',
    'current_speed': '13.4'},
   {'obs_time': '2022-02-25 00:30:00',
    'current_direct': '236.0',
    'current_speed': '11.3'},
   {'obs_time': '2022-02-25 01:00:00',
    'current_direct': '250.0',
    'current_speed': '14.5'},
   {'obs_time': '2022-02-25 01:30:00',
    'current_direct': '268.0',
    'current_speed': '17.4'},
   {'obs_time': '2022-02-25 02:00:00',
    'current_direct': '278.0',
    'current_speed': '18.1'},
   {'obs_time': '2022-02-25 02:30:00',
    'current_direct': '282.0',
    'current_speed': '20.8'},
   {'obs_time': '2022-02-25 03:00:00',
    'current_direct': '271.0',
    'current_speed': '23.9'},
   {'obs_time': '2022-02-25 03:30:00',
    'current_direct': '294.0',
    'current_speed': '17.1'},
   {'obs_time': '2022-02-25 04:00:00',
    'current_direct': '269.0',
    'current_speed': '29.0'},
   {'obs_time': '2022-02-25 04:30:00',
    'current_direct': '313.0',
    'current

In [86]:
df = spark.createDataFrame(response["result"]["data"])
df1 = spark.createDataFrame(response1["result"]["data"])

In [87]:
df.show(5)

+-------------------+----------+
|        record_time|water_temp|
+-------------------+----------+
|2022-02-25 00:00:00|     17.21|
|2022-02-25 00:30:00|     17.21|
|2022-02-25 01:00:00|     17.21|
|2022-02-25 01:30:00|     17.21|
|2022-02-25 02:00:00|     17.21|
+-------------------+----------+
only showing top 5 rows



In [88]:
df1.show(5)

+--------------+-------------+-------------------+
|current_direct|current_speed|           obs_time|
+--------------+-------------+-------------------+
|         224.0|         13.4|2022-02-25 00:00:00|
|         236.0|         11.3|2022-02-25 00:30:00|
|         250.0|         14.5|2022-02-25 01:00:00|
|         268.0|         17.4|2022-02-25 01:30:00|
|         278.0|         18.1|2022-02-25 02:00:00|
+--------------+-------------+-------------------+
only showing top 5 rows



In [89]:
output_df1 = df.join(df1,df.record_time==df1.obs_time).select(df.record_time, df.water_temp, df1.current_direct, df1.current_speed)

In [90]:
/var/lib/docker/volumes/<volume-name>/var/lib/docker/volumes/<volume-name>output_df1 = output_df1.sort(asc("record_time"))
output_df1.show(5)

+-------------------+----------+--------------+-------------+
|        record_time|water_temp|current_direct|current_speed|
+-------------------+----------+--------------+-------------+
|2022-02-25 00:00:00|     17.21|         224.0|         13.4|
|2022-02-25 00:30:00|     17.21|         236.0|         11.3|
|2022-02-25 01:00:00|     17.21|         250.0|         14.5|
|2022-02-25 01:30:00|     17.21|         268.0|         17.4|
|2022-02-25 02:00:00|     17.21|         278.0|         18.1|
+-------------------+----------+--------------+-------------+
only showing top 5 rows



In [91]:
output_df1 = output_df1.withColumn('water_temp', output_df1['water_temp'].cast(DoubleType()))\
    .withColumn('current_direct', output_df1['current_direct'].cast(DoubleType()))\
    .withColumn('record_time', output_df1['record_time'].cast('timestamp'))\
    .withColumn('current_speed', output_df1['current_speed'].cast(DoubleType()))
output_df1 = output_df1.withColumn('id',lit(response["result"]["meta"]["obs_post_id"]))

In [92]:
output_df1.show(5)

+-------------------+----------+--------------+-------------+-------+
|        record_time|water_temp|current_direct|current_speed|     id|
+-------------------+----------+--------------+-------------+-------+
|2022-02-25 00:00:00|     17.21|         224.0|         13.4|KG_0021|
|2022-02-25 00:30:00|     17.21|         236.0|         11.3|KG_0021|
|2022-02-25 01:00:00|     17.21|         250.0|         14.5|KG_0021|
|2022-02-25 01:30:00|     17.21|         268.0|         17.4|KG_0021|
|2022-02-25 02:00:00|     17.21|         278.0|         18.1|KG_0021|
+-------------------+----------+--------------+-------------+-------+
only showing top 5 rows



In [93]:
output_df1.write.format('jdbc').options(
    url='jdbc:mysql://'+ip+'/weather',
    driver='com.mysql.jdbc.Driver',
    dbtable='KG_0021',
    user='mdgome',
    password='Rlawjdals1!').mode('append').save()

In [38]:
spark.stop()